# Table of Contents
- [Data gathering and preparation](#prep-start)
    - [FRED data](#prep-fred)
    - [FDIC data](#prep-fdic)
    - [Merging FRED and FDIC data](#prep-merge)
- [Initial data exploration](#explo-start)
    - [Outlier detection](#explo-outlier)
    - [Summary statistics](#explo-summary)
    - [Correlation heatmap](#explo-corr)
- [Unsupervised modelling exploration](#unsupervised)
- [Supervised modelling and analysis](#supervised)
    - [Results and interpretation](#supervised-results)

In [1]:
import requests

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pyarrow.feather import read_feather, write_feather

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 50)

In banking, forecasting deposit movements helps to manage cash flows and inform lending and investing decisions. Our goal is specifically to forecast bank deposit size using key macroeconomic indicators and bank-level data.

[Github repository](https://github.com/hughdingb58/deposit_size_prediction)

<a id="prep-start"></a>
# Data gathering and preparation

<a id="prep-fred"></a>
## FRED data

<a id="prep-fdic"></a>
## FDIC data

FDIC data is gathered from its [API](https://banks.data.fdic.gov/docs/).

In [3]:
fdic_url = "https://banks.data.fdic.gov/api/"

### Bank metadata

In [4]:
institutions_url = fdic_url + "institutions"

institutions_params = {
    "filters": "ACTIVE:1",
    "sort_by": "OFFICES",
    "sort_order": "DESC",
    "limit": 10000,
    "format": "json"
}

# Get list of institutions from API
res = requests.get(institutions_url, params = institutions_params)
institutions_data = res.json()["data"]

In [5]:
# Read data into list
institutions_list = []
for row in institutions_data:
    curr_bank = pd.DataFrame(row["data"], index = [0])
    institutions_list.append(curr_bank)

# Bind list into table
institutions_full = pd.concat(institutions_list, axis = 0).reset_index(drop = True)

In [6]:
# Keep only relevant columns
institutions = institutions_full[["NAMEHCR", "STNAME", "CITY", "ZIP"]].drop_duplicates()
institutions = institutions[institutions["NAMEHCR"] != ""].reset_index(drop = True)
institutions["ZIP"] = institutions["ZIP"].astype(str)
institutions.head()

,NAMEHCR,STNAME,CITY,ZIP
0,JPMORGAN CHASE&CO,Ohio,Columbus,43240
1,WELLS FARGO&COMPANY,South Dakota,Sioux Falls,57104
2,BANK OF AMERICA CORP,North Carolina,Charlotte,28202
3,PNC FINL SERVICES GROUP INC,Delaware,Wilmington,19801
4,U S BCORP,Ohio,Cincinnati,45202


In [7]:
print("Number of active institutions: " + str(institutions.shape[0]))

Number of active institutions: 3816


### Financial data

In [8]:
fields = [
    "NAMEHCR", # Bank name
    "ZIP", # Bank location
    "REPDTE", # Reporting date
    "ASSET", # Total assets
    "NETINCQ", # Quarterly net income
    "NETINCQR", # Quarterly net income ratio
    "OFFOA", # Number of US offices
    "OFFFOR", # Number of foreign offices
    "ROAQ", # Return on assets
    "ROEQ", # Quarterly return on equity
    "EINTXQA", # Total interest expense quarterly
    "EINTXQR", # Total interest expense quarterly ratio
    "NONIXQA", # Total interest expense quarterly
    "NONIXQR", # Total interest expense quarterly ratio
    "DEP", # Total deposits
    "DEPR" # Total deposits ratio
]

fields_str = ",".join(fields)
fields_sum_str = ",".join(fields[3:])
print(fields_str)
print(fields_sum_str)

NAMEHCR,ZIP,REPDTE,ASSET,NETINCQ,NETINCQR,OFFOA,OFFFOR,ROAQ,ROEQ,EINTXQA,EINTXQR,NONIXQA,NONIXQR,DEP,DEPR
ASSET,NETINCQ,NETINCQR,OFFOA,OFFFOR,ROAQ,ROEQ,EINTXQA,EINTXQR,NONIXQA,NONIXQR,DEP,DEPR


In [9]:
financials_url = fdic_url + "financials"

financials_list = []

# Loop through banks
for i in range(institutions.shape[0]):
    institution_row = institutions.iloc[i]
    financials_params = {
        "filters": 'NAMEHCR:"' + str(institution_row["NAMEHCR"]) + '",ZIP:"' + str(institution_row["ZIP"]) + '"',
        "fields": fields_str,
        "sort_by": "REPDTE",
        "sort_order": "DESC",
        "limit": 10000,
        "agg_term_fields": "NAMEHCR,ZIP,REPDTE",
        "agg_sum_fields": fields_sum_str,
        "format": "json"
    }

    # Get list of institutions from API
    res = requests.get(financials_url, params = financials_params)
    financials_data = res.json()["data"]
    
    # Read data into list
    for row in financials_data:
        curr_bank = pd.DataFrame(row["data"], index = [0])
        financials_list.append(curr_bank)

    # To keep track of progress
    if (i % 50) == 0:
        print("row " + str(i) + ": " + str(institution_row["NAMEHCR"]))


row 0: JPMORGAN CHASE&CO
row 50: RENASANT CORP
row 100: PINNACLE BCORP INC
row 150: SUMMIT FINANCIAL GROUP INC
row 200: FIRST OF LONG ISLAND CORP THE
row 250: BANK OF NY MELLON CORP THE
row 300: ENTERPRISE BCORP INC
row 350: FRANKLIN FINL SERVICES CORP
row 400: CONSUMERS BCORP INC
row 450: ROYAL BANCSHARES INC
row 500: ALLIANCE BANCSHARES INC
row 550: TENNESSEE STATE BANCSHARES INC
row 600: SOUTHEAST BANCSHARES INC
row 650: CHINA INVESTMENT CORP
row 700: DECATUR INVESTMENT INC
row 750: DAIRY STATE BCORP INC
row 800: MONTGOMERY BCORP INC
row 850: VISION BANCSHARES INC
row 900: ORION BCORP INC
row 950: FIRST BCORP OF INDIANA INC
row 1000: MID-MISSOURI HOLDING CO INC
row 1050: MBT BANCSHARES INC
row 1100: CITIZENS CORP
row 1150: NORTHERN WI BK HOLDING CO INC
row 1200: FARMERS&MERCHANTS BANCSHARES
row 1250: METROPOLITAN BANK HOLDING CORP
row 1300: HIGH POINT FINL SERVICES INC
row 1350: ALLIANCE BCORP
row 1400: FIVE STAR BCORP
row 1450: TRUNORTH BCORP MHC
row 1500: MARION BANCSHARES INC
row

In [10]:
# Bind list into table
financials_data = pd.concat(financials_list, axis = 0).reset_index(drop = True)
financials_data["ZIP"] = financials_data["ZIP"].astype(str)

In [11]:
financials = pd.merge(institutions, financials_data, on = ["NAMEHCR", "ZIP"])
# Set more descriptive column names
financials.columns = ["name", "state", "city", "zip", "date", "total_assets", "roa_quarterly", "net_income_quarterly_ratio",
                      "total_interest_expense_quarterly", "total_deposits", "non_interest_expense_quarterly", "roe_quarterly",
                      "offices_foreign", "net_income", "offices_us", "total_deposits_ratio",
                      "total_interest_expense_quarterly_ratio", "non_interest_expense_quarterly_ratio", "id"]
# Clean date column
financials["date"] = pd.to_datetime(financials["date"], format = "%Y%m%d")
financials.sample(n = 10, random_state = 0)

,name,state,city,zip,date,total_assets,roa_quarterly,net_income_quarterly_ratio,total_interest_expense_quarterly,total_deposits,non_interest_expense_quarterly,roe_quarterly,offices_foreign,net_income,offices_us,total_deposits_ratio,total_interest_expense_quarterly_ratio,non_interest_expense_quarterly_ratio,id
400383,CITY NATIONAL BANKCORP INC,Illinois,Metropolis,62960,1994-09-30,87617,1.43,1.425520,2388,74030,1680,10.63,0,307,0,84.492735,2.772103,1.950223,3814_19940930
407396,CLOVIS BANCSHARES INC,New Mexico,Clovis,88101,2006-06-30,92041,1.17,1.170686,1632,82911,2356,14.05,0,250,0,90.080508,1.910560,2.758136,57022_20060630
120691,CARROLL FINANCIAL SERVICES INC,Tennessee,Huntingdon,38344,2004-03-31,153863,1.25,1.245555,2120,134731,4172,11.99,0,475,0,87.565562,1.389777,2.734975,2380_20040331
20806,HANMI FINANCIAL CORP,California,Los Angeles,90010,2006-09-30,3736803,2.08,2.079663,109048,2975219,75424,14.14,0,19128,0,79.619370,2.964020,2.050091,24170_20060930
339845,HOMETOWN BANCSHARES INC,Alabama,Hamilton,35570,2007-03-31,45092,0.42,0.420605,1416,38467,1400,2.95,0,47,0,85.307815,3.167962,3.132166,57569_20070331
122982,WINTRUST FINANCIAL CORP,Illinois,Lake Forest,60045,1998-06-30,390153,0.95,0.949949,15048,362513,7320,16.29,0,951,0,92.915600,3.757843,1.827978,27589_19980630
372650,WB&T BANKSHARES INC,Georgia,Brunswick,31520,2022-12-31,214753,1.41,1.411891,1708,198243,4988,23.99,0,737,0,92.312098,0.818015,2.388912,16283_20221231
305086,WILSON BANCSHARES INC,Missouri,Weston,64098,1988-03-31,29521,1.40,1.398798,1468,26729,956,18.8,0,112,0,90.542326,4.583561,2.984935,1418_19880331
259229,FIRST BCORP INC,Kentucky,Campbellsville,42718,1994-09-30,60202,1.10,1.104063,1576,50830,2000,10.53,0,166,0,84.432411,2.620487,3.325491,15675_19940930
35271,CITIZENS BANCSHARES CORP,South Carolina,Olanta,29114,1996-03-31,97748,1.81,1.814723,3704,84861,2960,16.5,0,440,0,86.816099,3.819167,3.052034,15932_19960331


Most data values look reasonable, but the ID doesn't seem meaningful, and the office counts don't seem to be properly filled in, so we will exclude those.

In [12]:
# Remove ID column
financials = financials.drop(["id", "offices_foreign", "offices_us"], axis = 1)
# Cache values
write_feather(financials, "fdic_financials.feather")

In [14]:
financials = read_feather("fdic_financials.feather")

<a id="prep-merge"></a>
## Merging FRED and FDIC data

In [15]:
dataset_full = financials

<a id="explo-start"></a>
# Initial data exploration

<a id="explo-outlier"></a>
## Outlier detection

<a id="explo-summary"></a>
## Summary statistics

<a id="explo-corr"></a>
## Correlation heatmap

<a id="unsupervised"></a>
# Unsupervised modelling exploration

<a id="supervised"></a>
# Supervised modelling and analysis

<a id="supervised-results"></a>
## Results